In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Refs.

1. Introduction to the theory of neuronal computation, Hertz, Krogh, Palmer (1991)

2. https://machinelearningmastery.com/implement-perceptron-algorithm-scratch-python/

3. https://towardsdatascience.com/perceptron-explanation-implementation-and-a-visual-example-3c8e76b4e2d1

## Teoría

### Múltiples capas y neuronas de salida

Consideramos un perceptrón de $m+1$ capas y de $n_l$ neuronas en la $l$-ésima capa, siendo $n_0$ el número de neuronas en la capa de entrada y $n_m$ el número de neuronas en la capa de salida.

La componente $V_{i}^{(l)}\in\mathbb{R}$ representa el estado de la $i$-ésima neurona en la $l$-ésima capa,
la componente $w_{ji}^{(l)}\in\mathbb{R}$ el peso sináptico saliendo de la $i$-ésima neurona en la $(l-1)$-ésima capa y entrando a la $j$-ésima neurona de la $l$-ésima capa.
De esta manera, el estado $V_{j}^{(l+1)}$ de la $j$-ésima neurona de la $(l+1)$-ésima capa viene determinado por

$$ V_{j}^{(l+1)} = g(h_{j}^{l+1}) \;\;\; (1)$$

donde

$$ h_{j}^{(l+1)} := \sum_i w_{ji}^{(l+1)}V_{i}^{(l)} \;\;\; (2)$$

Asuminos que en cada capa $l<m$ existe una neurona de estado $V_{0}^{(l)}=1$ con el fin de implementar el truquito que nos permite remplazar el umbral en cada neurona por un peso sináptico.

Notar que, con excepción de las neuronas de entrada en la capa $l=0$, todas las neuronas usan la misma función activación $g$.
Existen muchas funciones de activación, pero nos enfocaremos en usar

\begin{eqnarray}
g(x)
&:=& \tanh(\beta x) \\
&=& \frac{e^{\beta x}-e^{-\beta x}}{e^{\beta x}+e^{-\beta x}} \;\;\; (3) \\
\end{eqnarray}

cuya derivada convenientemente satisface $g' = \beta(1-g^2)$.

#### Entrenamiento

Consideraremos una serie de datos de $\mu=1,...,p$ muestras, de entradas $\xi_{\mu i}$ y salidas $\zeta_{\mu j}$ en $\mathbb{R}$.
Es decir, $x_{\mu i}$ representa el valor de entrada de la $i$-ésima neurona en la capa $l=0$ y $\zeta_{\mu j}$ el valor pretendido de salida en la $j$-ésima neurona de la capa $l=m$.

El objetivo es entrenar los pesos sinápticos representados por $w$, de manera que la salida 

$$ V_{\mu j}^{(m)} =: O_{\mu j} \approx \zeta_{\mu j}\;\;\; (4) $$ 

cuando la entrada es

$$ V_{\mu i}^{(0)} = \xi_{\mu i} \;\;\; (5) $$

para todo $i=1,...,n_0$, $j=1,...,n_m$ y muestra $\mu=1,...,p$.
Aquí, $V_{\mu j}^{(m)}$ es el valor que adopta la $j$-ésima neurona en la capa $m$ cuando la red es expuesta a la $\mu$-ésima entrada.

Formalmente, buscaremos minimizar el error cuadrático

\begin{eqnarray}
E := \sum_{\mu j} \frac{1}{2}(\zeta_{\mu j}-O_{\mu j})^2 \;\;\; (6)
\end{eqnarray}

con respecto a $w$.
Aquí, debemos recordar que $V_{\mu j}^{(m)}$ depende de los pesos sinápticos $w$ y de las entradas $x$ (ver Ecs. (1-5)).

Como algoritmo de minimización, utilizaremos una versión del método de descenso por el gradiente llamado **back-propagation**.
Combiene simplificar notación y definir el operador derivada parcial $\partial_x$ por $\partial_x f := \frac{\partial }{\partial x}f$ para cualquier funcion $f:\mathbb{R}\to \mathbb{R}$.
La idea nace de calcular el gradiente de $E$ respecto de $w$, capa por capa.
Comenzamos por calcular componentes del gradiente correspondientes a pesos sinápticos de la última capa

\begin{eqnarray}
\partial_{w_{rs}^{(m)}}E
&=&\sum_{\mu j} \frac{1}{2}\partial_{w_{rs}^{(m)}}(\zeta_{\mu j}-O_{\mu j})^2\\
&=&\sum_{\mu j} \frac{1}{2}\partial_{w_{rs}^{(m)}}(\zeta_{\mu j}-V_{\mu j}^{(m)})^2\\
&=&\sum_{\mu j} (\zeta_{\mu j}-V_{\mu j}^{(m)})\partial_{w_{rs}^{(m)}}(\zeta_{\mu j}-V_{\mu j}^{(m)}) \;\;\; (7)\\
&=&-\sum_{\mu j}(\zeta_{\mu j}-V_{\mu j}^{(m)})\partial_{w_{rs}^{(m)}}V_{\mu j}^{(m)}\\
&=&\sum_{\mu j}(V_{\mu j}^{(m)}-\zeta_{\mu j})\partial_{w_{rs}^{(m)}}g(h_{\mu j}^{(m)})\\
&=&\sum_{\mu j}(V_{\mu j}^{(m)}-\zeta_{\mu j})g'(h_{\mu j}^{(m)})\partial_{w_{rs}^{(m)}}h_{\mu j}^{(m)}\\
&=&\sum_{\mu j}(V_{\mu j}^{(m)}-\zeta_{\mu j})g'(h_{\mu j}^{(m)})\partial_{w_{rs}^{(m)}}\sum_i w_{ji}^{(m)}V_{\mu i}^{(m-1)}\\
&=&\sum_{\mu ji}(V_{\mu j}^{(m)}-\zeta_{\mu j})g'(h_{\mu j}^{(m)})\big(\delta_{mm}\delta_{rj}\delta_{si}V_{\mu i}^{(m-1)}+w_{ji}^{(m)}\partial_{w_{rh}^{(m)}}V_{\mu i}^{(m-1)}\big)\\
&=&\sum_{\mu ji}(V_{\mu j}^{(m)}-\zeta_{\mu j})g'(h_{\mu j}^{(m)})\big(\delta_{rj}\delta_{si}V_{\mu i}^{(m-1)}+w_{ji}^{(m)}0\big)\\
&=&\sum_{\mu }(V_{\mu r}^{(m)}-\zeta_{\mu r})g'(h_{\mu r}^{(m)})V_{\mu s}^{(m-1)}\\
\end{eqnarray}

donde usamos que $\partial_{w_{rs}^{(m)}}V_{i}^{(m-1)}=0$ porque $V_{i}^{(l)}$ no depende de $w_{rs}^{(f)}$ para todo $f>l$ como puede deducirse de mirar la figura

<img src="fig1.png" width="400">

Luego, introduciendo

$$ d^{(m)}_{\mu r} := (O_{\mu r}-\zeta_{\mu r})g'(h_{\mu r}^{(m)}) \;\;\; (8)$$

reescribimos el anterior resultado como

\begin{eqnarray}
\partial_{w_{rs}^{(m)}}E
&=&\sum_{\mu }d^{(m)}_{\mu r}V_{\mu s}^{(m-1)} \;\;\; (9)\\
\end{eqnarray}

la cual convenientemente constituye una expresión en donde no aparecen derivadas explícitamente.

Luego, nos interesa calcular las componentes del gradiente para pesos sinápticos de alguna capa oculta arbitrara
\begin{eqnarray}
\partial_{w_{rs}^{(m-p)}}E
&=&\sum_{\mu j}(V_{\mu j}^{(m)}-\zeta_{\mu j})g'(h_{\mu j}^{(m)})\partial_{w_{rs}^{(m-p)}}h_{\mu j}^{(m)} \;\;\; (10) \\
&=&\sum_{\mu j}d^{(m)}_{\mu r}\partial_{w_{rs}^{(m-p)}}\sum_i w_{ji}^{(m)}V_{\mu i}^{(m-1)}\\
&=&\sum_{\mu ji}d^{(m)}_{\mu r}w_{ji}^{(m)}\partial_{w_{rs}^{(m-p)}}V_{\mu i}^{(m-1)}\\
&=&\sum_{\mu ji}d^{(m)}_{\mu r}w_{ji}^{(m)}\partial_{w_{rs}^{(m-p)}}g(h_{\mu i}^{(m-1)})\\
&=&\sum_{\mu ji}d^{(m)}_{\mu r}w_{ji}^{(m)}g'(h_{\mu i}^{(m-1)})\partial_{w_{rs}^{(m-p)}}h_{\mu i}^{(m-1)}\\
&=&\sum_{\mu i}g'(h_{\mu i}^{(m-1)})\big(\sum_j d^{(m)}_{\mu r}w_{ji}^{(m)}\big)\partial_{w_{rs}^{(m-p)}}h_{\mu i}^{(m-1)}\\
\end{eqnarray}

Aquí, conviene introducir una definición análoga a la de los $d^{(m)}_{\mu i}$, pero para la capa $(m-1)$-ésima

$$ d^{(m-1)}_{\mu i} := g'(h_{\mu i}^{(m-1)})\sum_j d^{(m)}_{\mu r}w_{ji}^{(m)} \;\;\; (11) $$

así podemos reescribir la expresión de la última línea en la Ec. (10) como

\begin{eqnarray}
\partial_{w_{rs}^{(m-p)}}E
&=&\sum_{\mu i}g'(h_{\mu i}^{(m-1)})\big(\sum_j d^{(m)}_{\mu r}w_{ji}^{(m)}\big)\partial_{w_{rs}^{(m-p)}}h_{\mu i}^{(m-1)} \;\;\; (12)\\
&=&\sum_{\mu i}d^{(m-1)}_{\mu i}\partial_{w_{rs}^{(m-p)}}h_{\mu i}^{(m-1)}\\
\end{eqnarray}

Si bien en esta expresión aún aparecen derivadas explícitas, procederemos por inducción para eliminarlas.
Para ello, definimos

$$ d^{(m-f)}_{\mu i} := g'(h_{\mu i}^{(m-f)})\sum_j d^{(m-f+1)}_{\mu r}w_{ji}^{(m-f+1)} \;\;\; (13) $$

y asumimos la hipótesis inductiva

$$ \partial_{w_{rs}^{(m-p)}}E = \sum_{\mu i}d^{(m-f)}_{\mu i}\partial_{w_{rs}^{(m-p)}}h_{ki}^{(m-f)} \;\;\; (14) $$

para cualquier $f$ tal que $0<f<p$.
En particular, notar que la hipótesis inductiva vale para $f=1$, tal como se deduce en la Ec. (12).

Luego, si la hipótesis inductiva vale para $f$, entonces el siguiente cálculo

\begin{eqnarray}
\partial_{w_{rs}^{(m-p)}}E
&=&\sum_{\mu i}d^{(m-f)}_{\mu i}\partial_{w_{rs}^{(m-p)}}h_{\mu i}^{(m-f)} \;\;\; (15)\\
&=&\sum_{\mu i}d^{(m-f)}_{\mu i}\partial_{w_{rs}^{(m-p)}}\sum_j w_{ij}^{(m-f)}V_{\mu j}^{(m-f-1)}\\
&=&\sum_{\mu ij}d^{(m-f)}_{\mu i}w_{ij}^{(m-f)}\partial_{w_{rs}^{(m-p)}}V_{\mu j}^{(m-f-1)}\\
&=&\sum_{\mu ij}d^{(m-f)}_{\mu i}w_{ij}^{(m-f)}\partial_{w_{rs}^{(m-p)}}g(h_{\mu j}^{(m-f-1)})\\
&=&\sum_{\mu ij}d^{(m-f)}_{\mu i}w_{ij}^{(m-f)}g'(h_{\mu j}^{(m-f-1)})\partial_{w_{rs}^{(m-p)}}h_{\mu j}^{(m-f-1)}\\
&=&\sum_{\mu j}g'(h_{\mu j}^{(m-f-1)})\big(\sum_i d^{(m-f)}_{\mu i}w_{ij}^{(m-f)}\big)\partial_{w_{rs}^{(m-p)}}h_{\mu j}^{(m-f-1)}\\
&=&\sum_{\mu j}d^{(m-f-1)}_{\mu j}\partial_{w_{rs}^{(m-p)}}h_{\mu j}^{(m-f-1)}\\
\end{eqnarray}

muestra que la hipótesis inductiva vale para $f+1$.

La hipótesis inductiva puede aplicarse recursivamente hasta eventualmente llegar al caso $f=p$ en donde deja de valer, pero en cambio se obtiene

\begin{eqnarray}
\partial_{w_{rs}^{(m-p)}}E
&=&\sum_{\mu i}d^{(m-p)}_{\mu i}\partial_{w_{rs}^{(m-p)}}\sum_j w_{ij}^{(m-p)}V_{\mu j}^{(m-q-1)} \;\;\; (16)\\
&=&\sum_{\mu ij}d^{(m-p)}_{\mu i}\delta_{ri}\delta_{sj}V_{\mu j}^{(m-p-1)}\\
&=&\sum_{\mu }d^{(m-p)}_{\mu r}V_{\mu s}^{(m-p-1)}\\
\end{eqnarray}

la cual es una expresión en donde no aparece explícitamente derivada alguna, y es el resultado al cuál queríamos llegar.

En otras palabras, vemos que puede obtenerse una expresión en donde no aparecen derivadas explícitas para cada componente $\partial_{w_{rs}^{(l)}}E$ del gradiente.
Por otro lado, es importante resaltar que estas expresiones dependen de los $d^{(l)}_{\mu r}$, los cuales están recursivamente determinados por la Ec. (13) en donde tampoco aparecen derivadas explíticas, salvo las de la función activación $g$.

#### Algoritmo

El principal objetivo es minimizar el error $E$ usando el algoritmo de descenso por el gradiente, iterando

$$ w^{(l)}_{ji}(t+1) = w^{(l)}_{ji}(t) - \eta \partial_{w^{(l)}_{ji}}E(t) \;\;\; (17)$$

sobre $t$ para todo $l$ y $ji$.
En esta notebook, iniciaremos la iteración desde una condición inicial aleatoria $w^{(l)}_{ji}(t=0) \sim \mathcal{N}$ donde $\mathcal{N}$ representa una distribución normal de media 0 y varianza 1.

Para calcular los $\partial_{w^{(l)}_{ji}}e$ necesitamos calcular los $V^{(l)}_{\mu i}$ y $d^{(l)}_{\mu i}$.
Ambos requieren del cálculo de los $h^{(l)}_{\mu i}$.
Observando la Ec. (13), nos damos cuenta que es necesario calcular primero los $d^{(m)}_{\mu i}$, luego los $d^{(m-1)}_{\mu i}$ y así hasta poder calcular los $d^{(1)}_{\mu i}$.

Luego de inicializar

$$ V^{(0)}_{\mu i} = \xi_{\mu i} $$

el cálculo se puede realizar en dos fases. 
Primero la fase **forward**, en donde calculamos los $V^{(l)}_{\mu i}$ y $h^{(l)}_{\mu i}$ capa por capa en orde creciente en $l$.
Más precisamente

   
1. Iterando sobre $l=1,2,...,m$, calculamos para cada $l$:

    i. $h^{(l)}_{\mu i} = \sum_j w^{(l)}_{ij} V^{(l-1)}_{\mu j}$
    
    ii. $V^{(l)}_{\mu i} = g(h^{(l)}_{\mu i})$

Luego la fase **backwards**, en donde calculamos los $d^{(l)}_{ki}$, $\partial_{w^{(l)}_{ji}}E$ y actualizamos los $w^{(l)}_{ji}$ en orden decreciente en $l$.
Más precisamente

1. Calculamos

    i. $ d_{\mu i}^{(m)} = g'(h^{(m)}_{\mu i})(V_{\mu i}^{(m)}-\zeta_{\mu i}) $

    y actualizamos

    ii. $ w^{(m)}_{ji} -= \eta \sum_{\mu} d^{(m)}_{\mu j} V^{(m-1)}_{\mu i} $

2. Luego, iterando sobre $l=m-1,m-2,...,1$, calculamos para cada $l$:

    i. $ d_{\mu i}^{(l)} = g'(h^{(l)}_{\mu i}) \sum_j d^{(l+1)}_{\mu j} w^{(l+1)}_{ji} $
    
    y actualizamos
    
    ii. $ w^{(l)}_{ji} -= \sum_{\mu} d^{(l)}_{\mu j} V^{(l-1)}_{\mu i} $

#### Minipráctico

**a)** Implemente un perceptron de $1+m=3$ capas, una de entrada de $n_0=2$ neuronas, una oculta de $n_1=2$ neuronas y una de salida de $n_2=1$ neurona, utilizando $g(x) = \tanh(\beta x)$ como función activación.

In [ ]:
beta=1.0
def g(x):
    return np.tanh(beta*x)
def dg(x):
    return beta*(1.0-g(x)**2)

class Perceptron:
    def __init__(self,n,x,z,epocas=100,eta=0.01,g=g,dg=dg):
        """
        n: N^(1+m) lista de tamaños de capas (sin incluir neuronas truquito)
        x: R^(q,n[0])
        z: R^(q,n[m])
        epocas: N
        beta: R
        eta: R
        """
        m=len(n)-1 # n[0],n[1],...,n[m]
        p,_=x.shape
        self.n=n
        self.m=m        
        self.beta=beta
        V = [None]*(1+m)
        h = [None]*(1+m)
        d = [None]*(1+m)
        w = [None]*(1+m)
        # Inicializamos pesos. Las primeras capas tienen en cuenta neuronas truquito. La ultima no.
        for l in range(1,m):
            w[l]=np.random.normal(size=(n[l]+1,n[l-1]+1))
        w[m]=np.random.normal(size=(n[m],n[m-1]+1))
        # Inicializamos activaciones, preactivaciones y diferencias
        V[0] = np.ones((p,n[0]+1)) # Inicializamos y agregamos truquito a V^0_ki
        V[0][:,1:] = x[:,:]
        # Entrenamos
        self.list_E=[]
        for t in range(epocas):
            # Forward
            for l in range(1,m+1):
                h[l] = np.tensordot(V[l-1],w[l],axes=([1],[1]))
                V[l] = np.vectorize(g)(h[l])
            # Calculamos error cuadratico y reportamos
            err = V[m]-z.reshape(V[m].shape)
            self.list_E.append(np.dot(err.flatten(),err.flatten()))
            # Backward
            d[m] = np.vectorize(dg)(h[m])*err
            w[m] -= eta*np.tensordot(d[m],V[m-1],axes=([0],[0]))
            for l in range(m-1,0,-1):
                d[l] = np.vectorize(dg)(h[l])*np.tensordot(d[l+1],w[l+1],axes=([1],[0]))
                w[l] -= eta*np.tensordot(d[l],V[l-1],axes=([0],[0]))
        # grabamos los pesos como miembro de la clase
        self.w=w
    def __call__(self,x):
        n=self.n
        m=self.m
        w=self.w
        V = [None]*(1+m)
        h = [None]*(1+m)
        V[0] = np.ones(n[0]+1)
        V[0][1:] = x
        for l in range(1,m+1):
            h[l] = np.dot(w[l],V[l-1])
            V[l] = np.vectorize(g)(h[l])
        return V[m]

**b)** Usar `scikit-learn.datasets.make_classification` para crear un dataset para clasificación con:

- 2 características (features)
- 2 clases
- 100 muestras
- sin redundancia
- 1 grupo (cluster) por clase

Grafique el dataset

In [ ]:
from sklearn.datasets import make_classification

n0 = 2  # n[0]
p = 100 # mu=0,1,...,p-1 donde p = número de muestras.
n_classes = 2 
x,y = make_classification(
    n_features=n0,
    n_classes=n_classes,
    n_samples=p,
    n_redundant=0,
    n_clusters_per_class=1
)

print("x.shape=",x.shape,"y.shape=",y.shape,sep="")

color = {0:'red',1:'blue',2:'green',3:'cyan'}
for k in range(x.shape[0]):
    plt.scatter([x[k,0]],[x[k,1]],c=color[y[k]])
plt.title("datos")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

**c)** Entrene el perceptron en el dataset generado

In [ ]:
p = Perceptron([2,2,1],x,y)
plt.plot(range(len(p.list_E)),p.list_E)

**d)** Grafique el resultado del entrenamiento

In [ ]:
# Ploteamos el resultado del entrenamiento
def heaviside(x):
    if x>0.5:
        return 1
    return 0

num_error = 0
for k in range(x.shape[0]):
    y_pred=p(x[k,:])[0]
    y_pred_bin = heaviside(y_pred)
    if y[k]!=y_pred_bin:
        num_error += 1
    #print("k=",k," y=",y[k]," y_pred_bin=",y_pred_bin," y_pred=",y_pred,sep="")
    c_pred = color[y_pred_bin]
    c_true = color[y[k]]
    plt.scatter([x[k,0]],[x[k,1]],color=c_pred,marker='.',edgecolors=c_true,linewidth=1,s=250)
print("num_error=",num_error,sep="")
    
xmin = np.min(x[:,0])
xmax = np.max(x[:,0])
xs = np.linspace(xmin,xmax,100)
ymax = np.max(x[:,1])
ymin = np.min(x[:,1])
plt.title("prediccion")

**e)** El problema de la compuerta **XOR**.

La compuerta XOR viene dada por

    0 0 -> 0
    0 1 -> 1
    1 0 -> 1    
    1 1 -> 0
    
Muestre que un perceptrón simple no puede aprender la compuerta XOR, mientras que un perceptrón con sólo una capa oculta de dos neuronas (i.e. una capa de entrada de dos neuronas, una oculta de dos neuronas y una de salida de 1 neurona) si puede.

In [ ]:
# Generamos los datos
x = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,1,1,0])
for k in range(x.shape[0]):
    plt.scatter([x[k,0]],[x[k,1]],c=color[y[k]])
plt.title("datos")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

In [ ]:
# Entrenamos el perceptron simple
p_simple = Perceptron([2,1],x,y,epocas=4000)
plt.plot(range(len(p_simple.list_E)),p_simple.list_E)

In [ ]:
for k in range(x.shape[0]):
    y_pred=p_simple(x[k,:])[0]
    y_pred_bin = heaviside(y_pred)
    print("x=",x[k,:]," y=",y[k]," y_pred_bin=",y_pred_bin," y_pred=",y_pred,sep="")
    c_pred = color[y_pred_bin]
    c_true = color[y[k]]
    plt.scatter([x[k,0]],[x[k,1]],color=c_pred,marker='.',edgecolors=c_true,linewidth=1,s=250)

In [ ]:
# Entrenamos el perceptron con una capa oculta.
p = Perceptron([2,2,1],x,y,epocas=4000)
plt.plot(range(len(p.list_E)),p.list_E)

In [ ]:
for k in range(x.shape[0]):
    y_pred=p(x[k,:])[0]
    y_pred_bin = heaviside(y_pred)
    print("x=",x[k,:]," y=",y[k]," y_pred_bin=",y_pred_bin," y_pred=",y_pred,sep="")
    c_pred = color[y_pred_bin]
    c_true = color[y[k]]
    plt.scatter([x[k,0]],[x[k,1]],color=c_pred,marker='.',edgecolors=c_true,linewidth=1,s=250)